# Expectation Maximization

The *Expectation Maximization (E-M) Algorithm* is an iterative approach to find maximum likelihood estimates for latent variables (since the things we want to maximize are only indirectly available). It is comprised of an estimation step, which tries to estimate the unknown variables, and a maximization step, which then tries to optimize the parameters of the model to better explain the data.

The unknown parameters are sometimes written as $\phi$ or $\Theta$, and we can call the latent, "nuisance," variables $J$, and the observed data $U$. So, from above, the process can be roughly seen as 
$$ \Theta^* = \operatorname*{argmax}_{\Theta} \sum_{J\in\mathcal{J}^n} P(\Theta, J|U) $$ 
Since this shows us maximizing the posterior probability of parameters $\Theta$ given our data and we are summing over $J$ in order to marginalize out our latent variables (Dellaert, 2002).

This process was first rigorously defined on the exponential family, where the probability density functions take the form 
$$ f(x|\phi) = b(x)\exp(\phi t(x)^T/a(\phi))$$

where $\phi$ is a $1\times r$ parameter vector and $t(x)$ is a $1\times r$ vector of sufficient statistics for the data. Our "natural parameter" for these exponential distributions is given by some $r\times r$ linear transformation. 

To run the \textbf{E-M} algorithm on this example, we first enter the expectation step, and take $t^{(p)}=E[t(x)|y,\phi^{(p)}]$  with the $(p)$ denoting the $p^{th}$ cycle of the algorithm, trying to estimate the vector of sufficient statistics for the exponential distribution. 

The maximization step, is then taking the equation $E[t(x)|y,\phi^{(p)}]=t^{(p)}$ and we call the solution to this equation $\phi^{(p+1)}$. We then plug in $\phi^{(p+1)}$ to the expectation step and keep iterating. 

This looks nice, and seems like it could work, but if you are anything like me, you might wonder how we decide when to stop. Possibly the coolest part of this algorithm is that it actually converges to a local maximum every time (Dempster *et al.* 1976). Since it always converges to a local maximum, it means that if we "guess" a decent parameter space to start off the algorithm, we will converge to what is likely to be a very solid set of estimates.

In Dempster, Laird, and Rubin's seminal paper *Maximum Likelihood via the 'EM' Algorithm*, they enumerate the process detailed above, proved the convergence, and later on, proposed that E-M could be used to in *finite clusters*. This foreshadowed the most common usage of the algorithm, clustering, or more specifically, dividing unlabeled data into nice clusters. For example, if we know that our raw data is comprised of unique groups represented by different probability distributions, we can use the E-M algorithm to change the parameters for the estimated distributions of these groups to maximize the probability that the data belongs to the proposed clustering.

## Gaussian Mixture Modeling

One of the most common usages of expectation maximization, and specifically clustering, is *Gaussian Mixture Modeling* (GMM) (Hasselblad 1966). This process is essentially assuming that each group you are trying to sort out is represented by a normal distribution. This is often a very convenient technique to use because things often actually do follow normal distributions because of the central limit theorem (everyone's favorite statistics theorem) and because once we have clusters that are normal, it is much easier to do inference on the clusters and talk about them with people who don't know as much about statistics because gaussian distributions are so well understood.

GMMs are used to observe clusters everywhere. They are used to create customer archetypes in retail, to better understand the different ways people shop, they are used in medical scenarios in order to identify types of tumors for cancer detection. From this little sample of use cases, it is pretty obvious that this is a really powerful (and pretty cool) use for a powerful algorithm.

## Example Code

In [ ]:
#importing necessary packages
from sklearn.mixture import GaussianMixture
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt 
import pandas as pd 
import numpy as np
from numpy import random
from urllib.request import urlopen
from bs4 import BeautifulSoup
import seaborn as sns 

To show how effective this algorithm can be, I am going to make a set of five blobs of data, each with a center, and then I will show how accurately the algorithm can cluster the data into the blobs that created the underlying data.


In [ ]:
#set a random seed so that we actually get clusters that kind of look like separate clusters
random.seed(195)
x, _ = make_blobs(n_samples=450, centers=5, cluster_std=1.84)
plt.figure(figsize=(8, 6))
plt.scatter(x[:,0], x[:,1])
plt.show() 

Now I'm going to fit the algorithm with the prior understanding that the data is made of five clusters of approximately normal data.


In [ ]:
#| echo: false
gm = GaussianMixture(n_components=5).fit(x)

gm.get_params() 

{'covariance_type': 'full',
 'init_params': 'kmeans',
 'max_iter': 100,
 'means_init': None,
 'n_components': 5,
 'n_init': 1,
 'precisions_init': None,
 'random_state': None,
 'reg_covar': 1e-06,
 'tol': 0.001,
 'verbose': 0,
 'verbose_interval': 10,
 'warm_start': False,
 'weights_init': None} 

This code here fits the model and lets it learn from the data, in the next plot, I will plot the centers that the data came up with, and then on the plot after that I will plot the boundaries for the clusters that the algorithm came up with.


In [ ]:
centers = gm.means_
plt.figure(figsize=(8, 6))
plt.scatter(x[:,0], x[:,1], label="data")
plt.scatter(centers[:,0], centers[:,1],c='r', label="centers")
plt.legend()
plt.show() 

In [ ]:
pred = gm.predict(x)

df = pd.DataFrame({'x':x[:,0], 'y':x[:,1], 'label':pred})
groups = df.groupby('label')

ig, ax = plt.subplots()
for name, group in groups:
    ax.scatter(group.x, group.y, label=name)

ax.legend()
plt.show() 

The accuracy is really fantastic. It plots the centers exactly where I would have, and then is able to pick which dots it belongs to which groups with great accuracy. 

Now if we consider a dataset with millions of points and possibly hundreds or thousands of dimensions, this allows for very sensitive anomaly detection for genetic disorders by clustering genes or proteins it helps us see just how helpful this could be.

## The NBA

![](nbaPic.jpeg){width="50%"}

Basketball as a sport is changing. Players like Stephen Curry have changed perceptions around what a point guard is supposed to do, Nikola Jokic is reinventing the center position, and some teams are playing with centers who are shorter than 6'5. Another even bigger change is the advent of extremely tall players playing seemingly positionless basketball, the trend started by players such as Kevin Durant and Kristaps Porzingis, and continued by younger players like Chet Holmgren and Victor Wembanyama.

People are playing basketball differently. To effectively understand the game, the old labels of point guard, shooting guard, center, power forward, and small forward don't seem to suffice, which means that we want to find new labels for positions in order to regroup players.

This seems to be a problem uniquely well suited to clustering. I plan on looking at a few things, how the clusters of players in the modern NBA compare to the positions that players are assigned to. Secondly, I am curious if the NBA has become more specialized, i.e. if there are more than five positions, and players are acquired and used for more specific purposes than in the past. To answer these questions, even just a little bit, I plan on clustering players from the 2022 season (stats acquired from Basketball Reference) and answering my first question using that data and model, and then taking in data from the 1986 season (that was fun and long enough ago to measure change) and looking at the difference in number of clusters for those seasons.


In [ ]:
# | output: false
year = 2022
url = 'https://www.basketball-reference.com/leagues/NBA_2022_per_game.html'
html = urlopen(url)
org_html = BeautifulSoup(html)
#this reads in data from an html file from basketball reference. I took this code from the documentation of an api for basketball data but I didn't like their functionality and so I just altered their source code to get my own thing :) the api was nba_api on PyPI

org_html.findAll('tr', limit=2) #this should get me the columns from the website.
headers = [th.getText() for th in org_html.findAll('tr', limit=2)[0].findAll('th')]
headers = headers[1:]
headers

In [ ]:
# | output: false
#now we want to turn this mess of text we ripped from html into a dataframe (thumbs up emoji)

rows = org_html.findAll('tr')[1:]
player_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]
data = pd.DataFrame(player_stats, columns = headers)
data.head() #YAYYYYY
data.tail()

There are a few instances of the same player showing multiple times in the dataframe since people were traded and played for different teams throughout the season, so I took the averages of all of their values to create a set of stats for the season for them.


In [ ]:
data=data.drop(columns=['Pos', 'Age', 'Tm', 'G', 'GS'], axis=1)

In [ ]:
#| echo: false
#| output: false
rep = data.Player.value_counts() ## Here is the list of such players
rep
pl = np.where([rep > 1])[1]
pl
grp = data.groupby(["Player"])          ## Grouping the same datapoints together. 
grp = grp.mean()  

So now that we have this data, I will take all of the features besides position, age, team, games played, and games started and will use them to create clusters so we can start to draw some conclusions. We should feel pretty good about modeling the clusters as gaussian since there are over eight hundred players that played in 2022 which means we should feel alright about assuming normality across each predictor.


In [ ]:
X = data.copy(deep=True)
X = X.drop("Player", axis = 1)

In [ ]:
model = GaussianMixture(n_components= 5, random_state=0).fit(X)